In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import lightgbm as lgbm
# import optuna.integration.lightgbm as lgbm

In [2]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'

In [3]:
def read_feather(filename):
    df = pd.read_feather(os.path.join(INPUT_DIR, filename))
    print(filename, df.shape)
    return df

In [4]:
train_df = read_feather('train.f')
test_df = read_feather('test.f')

train.f (1997595, 35)
test.f (390095, 30)


In [5]:
campaign_df = read_feather('campaign.f')
map_gv_df = read_feather('map_game_feed_native_video_assets.f')
ad_video_df = read_feather('advertiser_video.f')
ad_cvideo_df = read_feather('advertiser_converted_video.f')

campaign.f (14627, 4)
map_game_feed_native_video_assets.f (2796, 3)
advertiser_video.f (11707, 6)
advertiser_converted_video.f (198622, 8)


In [6]:
ad_cvideo_df = ad_cvideo_df.drop_duplicates(
    subset=['mst_advertiser_video_id', 
                   'mst_game_feed_id', 
                    'mst_video_template_id'], keep='last')

le = preprocessing.LabelEncoder()
le.fit(['vertical', 'horizontal'])
ad_cvideo_df['rectangle_type_id'] = le.transform(ad_cvideo_df['rectangle_type'])
ad_cvideo_df.drop(columns=['rectangle_type'], inplace=True)
print('ad_cvideo_df', ad_cvideo_df.shape)

ad_cvideo_df (107493, 8)


#### 連続変数の特徴量

In [7]:
def create_continuous_features(input_df):
    use_columns = [
        # 連続変数
        'first_login_interval',
        'max_login_interval', 
        'frequency', 
        'login_frequency', 
        'last_login_interval',
        'from_click',
    ]
    return input_df[use_columns].copy()

#### Category系の特徴量

In [8]:
def create_category_features(input_df):
    use_columns = [
        # category 系の id. label-encoding として使う
        'adnw_id',
        'adspot_id',
        'adspot_video_format_id',
        'game_feed_asset_type_id',
        'auction_type_id',
        'category_id',
        'header_bidding',
        'is_interstitial',
        'os',
#  os_version',
        'pos',
        'user_type_id'
    ]
    return input_df[use_columns].copy()

#### country_code

In [9]:
def create_countrycode(input_df):
    le = preprocessing.LabelEncoder()
    le.fit(['None', 'JP', 'US', 'KR'])
    return pd.DataFrame(le.transform(input_df['country_code'].fillna('None')), columns=['country'])

#### date系

In [10]:
def create_date_features(input_df):
    date_df = pd.DataFrame(pd.to_datetime(input_df['imp_at'], utc=True))
    date_df['imp_at'] = date_df['imp_at'].dt.tz_convert('Asia/Tokyo')
    date_df['day'] = date_df['imp_at'].dt.day
    date_df['hour'] = date_df['imp_at'].dt.hour
    date_df['total_minute'] = date_df['imp_at'].dt.hour*60+date_df['imp_at'].dt.minute
    date_df['dayofweek'] = date_df['imp_at'].dt.dayofweek
    date_df.drop(columns=['imp_at'], inplace=True)
    return date_df

#### campaign

In [11]:
def create_campaign_features(input_df):
    campaign = pd.merge(input_df[['campaign_id']], campaign_df, left_on='campaign_id', right_on='id', how='left')
    campaign.drop(columns=['campaign_id', 'id', 'mst_advertiser_id', 'mst_advertiser_order_id'], inplace=True)
    return campaign

#### map_game_feed_native_video_assets

In [12]:
def create_gamefeed_features(input_df):
    input_merge = pd.merge(input_df[['game_feed_id', 'advertiser_id', 'video_template_id']], map_gv_df, 
                           left_on='game_feed_id', right_on='mst_game_feed_id', how='left').drop(columns=['mst_game_feed_id'])
    
    horizontal = ad_video_df.copy()
    left_keys = ['horizontal_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    horizontal.columns = [f'horizontal_{c}' if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    vertical = ad_video_df.copy()
    left_keys = ['vertical_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    vertical.columns = [f'vertical_{c}' if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
    left_keys = [
        "horizontal_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    horizontal = ad_cvideo_df.copy()
    horizontal.columns = [f"horizontal_converted_{c}" if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    left_keys = [
        "vertical_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    vertical = ad_cvideo_df.copy()
    vertical.columns = [f"vertical_converted_{c}" if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
    input_merge.drop(columns=['game_feed_id', 'advertiser_id', 'video_template_id', 
                              'horizontal_mst_advertiser_video_id', 'vertical_mst_advertiser_video_id'], inplace=True)
    return input_merge

In [13]:
processors = [
    create_continuous_features,
    create_category_features,
    create_countrycode,
    create_date_features,
    create_campaign_features,
    create_gamefeed_features
]

In [14]:
def to_feature(input_df):
    out_df = pd.DataFrame()
    for func in processors:
        _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return out_df

In [15]:
train_feat_df = to_feature(train_df)
test_feat_df = to_feature(test_df)
y = train_df['target']

In [16]:
assert len(train_feat_df) == len(train_df)
assert len(test_feat_df) == len(test_df)

In [17]:
feature_count = len(train_feat_df.columns)

In [18]:
#del train_df, test_df

#### LightGBM による学習

In [19]:
from sklearn.metrics import average_precision_score

def pr_auc(y_pred, y_true):
    """lightGBM の round ごとに PR-AUC を計算する用"""
    score = average_precision_score(y_true.get_label(), y_pred)
    return "pr_auc", score, True

In [20]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'seed' : 0,
    'learning_rate':  0.1,
    'feature_pre_filter': False, 
    'lambda_l1': 6.419345380049121e-07, 
    'lambda_l2': 8.432801302426078, 
    'num_leaves': 212, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.9907178796872467, 
    'bagging_freq': 2, 
    'min_child_samples': 100
}

In [21]:
def train_lgbm(X, y, cv, params: dict, verbose=100):

    models = []
    # training data の target と同じだけのゼロ配列を用意
    # float にしないと悲しい事件が起こるのでそこだけ注意
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    num_boost_round=1000,
                                                    early_stopping_rounds=verbose,
                                                    feval=pr_auc,
                                                    verbose_eval=verbose)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

        print(f'Fold {i} PR-AUC: {average_precision_score(y_valid, y_pred):.4f}')

    score = average_precision_score(y, oof_pred)
    print('FINISHED \ whole score: {:.4f}'.format(score))
    return oof_pred, models, score

In [22]:
from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cv = list(fold.split(train_feat_df, y)) # もともとが generator なため明示的に list に変換する

In [23]:
def tuning_lgbm(X, y, cv, params, verbose=100):
    idx_train, idx_valid = cv[0]
    x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    lgbm_train = lgbm.Dataset(x_train, y_train)
    lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
    
    best_params, tuning_history = dict(), list()
    best = lgbm.train(params,
                                  lgbm_train,
                                  valid_sets=lgbm_eval,
                                  num_boost_round=1000,
                                  early_stopping_rounds=verbose,
                                  feval=pr_auc,
                                  verbose_eval=0)
    print('Best Params:', best.params)
    print('Best Iteration:', best.best_iteration)
    print('Best Score:', best.best_score)

In [24]:
# %%time
# tuning_lgbm(train_feat_df, y, cv, params=lgbm_param)

In [25]:
%%time
oof, models, score = train_lgbm(train_feat_df, y, cv, params=lgbm_param)

[LightGBM] [Info] Number of positive: 52392, number of negative: 1545684
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054653 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1863
[LightGBM] [Info] Number of data points in the train set: 1598076, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032784 -> initscore=-3.384468
[LightGBM] [Info] Start training from score -3.384468
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.107943	valid_0's pr_auc: 0.221944
[200]	valid_0's binary_logloss: 0.107582	valid_0's pr_auc: 0.223488
[300]	valid_0's binary_logloss: 0.107534	valid_0's pr_auc: 0.22359
Early stopping, best iteration is:
[245]	valid_0's binary_logloss: 0.107529	valid_0's pr_auc: 0.22383
Fold 0 PR-AUC: 0.2238
[LightGBM] [Info] Number of positive: 52392, n

In [26]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [27]:
feature_importance(models)

,name,imp_0,imp_1,imp_2,imp_3,imp_4,sum
40,horizontal_file_size,5319,3594,3750,5213,5648,23524
39,max_login_interval,5218,2897,2934,4697,5589,21335
37,last_login_interval,4891,2954,3193,4476,5032,20546
38,total_minute,4934,2698,2764,4185,4973,19554
36,first_login_interval,4277,2780,2772,3830,4497,18156
35,horizontal_converted_file_size,4119,2559,2610,3546,4307,17141
34,day,3401,1947,1917,2946,3467,13678
32,horizontal_duration,2318,1657,1640,2149,2432,10196
33,hour,2496,1176,1203,2173,2452,9500
31,login_frequency,1726,1289,1219,1625,1785,7644


In [28]:
%%time
pred_list = []
for model in models:
    print('best_iteration', model.best_iteration)
    pred = model.predict(test_feat_df, num_iteration = model.best_iteration)
    pred_list.append(pred)
    
pred = np.mean(pred_list, axis=0)

best_iteration 245
best_iteration 149
best_iteration 153
best_iteration 219
best_iteration 254
CPU times: user 1min 26s, sys: 1.31 s, total: 1min 28s
Wall time: 4.79 s


In [29]:
assert len(pred) == len(test_df)

In [30]:
sub_df = pd.DataFrame({ 'target': pred })
sub_df.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)

In [31]:
print('- feature={}'.format(feature_count))
print('- score={:.4f}'.format(score))

- feature=41
- score=0.2229


#### simple_3: tuning
- Wall time: 7min 19s
- feature=41
- score=0.2229
- publicLB= 0.1970

#### simple_2
- Wall time: 47min 46s
- feature= 41
- score= 0.214588
- publicLB= 0.1907

#### tuning
```
Best Params: {
    'objective': 'binary', 
    'boosting_type': 'gbdt', 
    'seed': 0, 
    'learning_rate': 0.1, 
    'feature_pre_filter': False, 
    'lambda_l1': 6.419345380049121e-07, 
    'lambda_l2': 8.432801302426078, 
    'num_leaves': 212, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.9907178796872467, 
    'bagging_freq': 2, 
    'min_child_samples': 100, 
    'num_iterations': 1000, 
    'early_stopping_round': 100
}
Best Iteration: 245
Best Score: 'pr_auc', 0.22382995580267329
```